In [1]:
import pandas as pd
import xarray as xr

sstdir='../data_src/oisst/daily/*.nc'
ds = xr.open_mfdataset(sstdir, parallel=True, chunks={'time': '500MB'})

msst =ds["sst"].resample(time='1MS').mean()
ds_msst = msst.compute()


In [2]:
print(ds_msst)

<xarray.DataArray 'sst' (time: 499, zlev: 1, lat: 720, lon: 1440)>
array([[[[       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         ...,
         [-1.7245162, -1.7245162, -1.7245162, ..., -1.7245162,
          -1.7245162, -1.7245162],
         [-1.7287096, -1.728387 , -1.728387 , ..., -1.7287096,
          -1.7287096, -1.7287096],
         [-1.7287096, -1.7287096, -1.7287096, ..., -1.7290322,
          -1.7290322, -1.7287096]]],


       [[[       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
...
          -1.7403333, -1.7203332],
         [-1.725    , -1.7596

In [3]:
#drop the 'zlev' dimension of ds_msst since it's not present in your Dataset:
ds_msst = ds_msst.squeeze('zlev')
#rename the 'time' dimension to 'date' to match the name in the Dataset:
ds_msst = ds_msst.rename({'time': 'date'})

In [4]:
ds_msst = ds_msst.drop('zlev')

In [5]:
import time
st = time.time()
dz1 = xr.open_zarr(
    'sst_anomaly.zarr', chunks='auto', 
    group='anomaly')
et = time.time()
print('Exe time: ', et-st, 'sec')
dz1

Exe time:  1.2584967613220215 sec


<xarray.Dataset>
Dimensions:      (date: 499, lat: 720, lon: 1440)
Coordinates:
  * date         (date) datetime64[ns] 1982-01-01 1982-02-01 ... 2023-07-01
  * lat          (lat) float64 -89.88 -89.62 -89.38 -89.12 ... 89.38 89.62 89.88
  * lon          (lon) float64 0.125 0.375 0.625 0.875 ... 359.4 359.6 359.9
Data variables:
    level        (date, lat, lon) int64 dask.array<chunksize=(2, 180, 360), meta=np.ndarray>
    sst_anomaly  (date, lat, lon) float64 dask.array<chunksize=(2, 180, 360), meta=np.ndarray>
    td           (date, lat, lon) float64 dask.array<chunksize=(2, 180, 360), meta=np.ndarray>

In [6]:
#Verify that the coordinates of your DataArray (dz) and Dataset (ds_msst) match
# Get coordinates from dz and ds_msst
dz_coords = dz1.coords
ds_msst_coords = ds_msst.coords

# Compare lat, lon and date coordinates
print((dz_coords['lat'] == ds_msst_coords['lat']).all())
print((dz_coords['lon'] == ds_msst_coords['lon']).all())
print((dz_coords['date'] == ds_msst_coords['date']).all())


<xarray.DataArray 'lat' ()>
array(True)
<xarray.DataArray 'lon' ()>
array(True)
<xarray.DataArray 'date' ()>
array(True)


In [7]:
# Convert date coordinates to Python set type and perform set difference
missing_dates_in_ds_msst = set(dz1.date.values) - set(ds_msst.date.values)
missing_dates_in_dz = set(ds_msst.date.values) - set(dz1.date.values)

print("Dates in dz but not in ds_msst:", missing_dates_in_ds_msst)
print("Dates in ds_msst but not in dz:", missing_dates_in_dz)

Dates in dz but not in ds_msst: set()
Dates in ds_msst but not in dz: set()


In [8]:
import numpy as np

# Compare lat, lon and date coordinates
print(np.allclose(dz_coords['lat'], ds_msst_coords['lat']))
print(np.allclose(dz_coords['lon'], ds_msst_coords['lon']))
# For 'date' comparison, we can still use '=='
print((dz_coords['date'] == ds_msst_coords['date']).all())

True
True
<xarray.DataArray 'date' ()>
array(True)


In [9]:
dz1['sst'] = ds_msst
print(dz1)

<xarray.Dataset>
Dimensions:      (date: 499, lat: 720, lon: 1440)
Coordinates:
  * date         (date) datetime64[ns] 1982-01-01 1982-02-01 ... 2023-07-01
  * lat          (lat) float32 -89.88 -89.62 -89.38 -89.12 ... 89.38 89.62 89.88
  * lon          (lon) float32 0.125 0.375 0.625 0.875 ... 359.4 359.6 359.9
Data variables:
    level        (date, lat, lon) int64 dask.array<chunksize=(2, 180, 360), meta=np.ndarray>
    sst_anomaly  (date, lat, lon) float64 dask.array<chunksize=(2, 180, 360), meta=np.ndarray>
    td           (date, lat, lon) float64 dask.array<chunksize=(2, 180, 360), meta=np.ndarray>
    sst          (date, lat, lon) float32 nan nan nan nan ... -1.799 -1.8 -1.8


In [10]:
chunk_size = dz1['level'].chunks
print(chunk_size)
chunk_size_date = dz1['level'].chunks[0][0]
chunk_size_lat = dz1['level'].chunks[1][0]
chunk_size_lon = dz1['level'].chunks[2][0]
print(chunk_size_date, chunk_size_lat, chunk_size_lon)

((2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1), (180, 180, 180, 180), (360, 360, 360, 360))
2 180 360


In [11]:
dz2 = dz1.chunk({'date': chunk_size_date, 'lat': chunk_size_lat, 'lon': chunk_size_lon})
print(dz2)

<xarray.Dataset>
Dimensions:      (date: 499, lat: 720, lon: 1440)
Coordinates:
  * date         (date) datetime64[ns] 1982-01-01 1982-02-01 ... 2023-07-01
  * lat          (lat) float32 -89.88 -89.62 -89.38 -89.12 ... 89.38 89.62 89.88
  * lon          (lon) float32 0.125 0.375 0.625 0.875 ... 359.4 359.6 359.9
Data variables:
    level        (date, lat, lon) int64 dask.array<chunksize=(2, 180, 360), meta=np.ndarray>
    sst_anomaly  (date, lat, lon) float64 dask.array<chunksize=(2, 180, 360), meta=np.ndarray>
    td           (date, lat, lon) float64 dask.array<chunksize=(2, 180, 360), meta=np.ndarray>
    sst          (date, lat, lon) float32 dask.array<chunksize=(2, 180, 360), meta=np.ndarray>


In [12]:
import zarr
import dask
#compressor = zarr.Blosc(cname='zstd', clevel=3, shuffle=2)
dask.config.set(scheduler='single-threaded')

In [13]:
dz2.to_zarr('mhw.zarr', mode='w', group='anomaly') #'sst_anomaly_test.zarr'


In [14]:
st = time.time()
dz = xr.open_zarr(
    'mhw.zarr', chunks='auto', 
    group='anomaly', decode_times=True)

et = time.time()
print('Exe time: ', et-st, 'sec')
dz

Exe time:  0.013476371765136719 sec


<xarray.Dataset>
Dimensions:      (date: 499, lat: 720, lon: 1440)
Coordinates:
  * date         (date) datetime64[ns] 1982-01-01 1982-02-01 ... 2023-07-01
  * lat          (lat) float32 -89.88 -89.62 -89.38 -89.12 ... 89.38 89.62 89.88
  * lon          (lon) float32 0.125 0.375 0.625 0.875 ... 359.4 359.6 359.9
Data variables:
    level        (date, lat, lon) int64 dask.array<chunksize=(2, 180, 360), meta=np.ndarray>
    sst          (date, lat, lon) float32 dask.array<chunksize=(2, 180, 360), meta=np.ndarray>
    sst_anomaly  (date, lat, lon) float64 dask.array<chunksize=(2, 180, 360), meta=np.ndarray>
    td           (date, lat, lon) float64 dask.array<chunksize=(2, 180, 360), meta=np.ndarray>